## Scraping US Hourly Wages

In [61]:
import urllib2
import lxml
from lxml import html
import cssselect
import pandas

### Part 1: Setting Up Methods and Data Structures

#### A helper method to get the document of a website. Requires an URL.

In [62]:
def scrapeSite(url):
    #request and connect to the site
    req = urllib2.Request(url, headers={'User-Agent' : "Magic Browser"}) 
    con = urllib2.urlopen( req )

    #read the connection as a document
    doc_text = con.read()
    doc = lxml.html.fromstring(doc_text)
    doc.make_links_absolute(url)
    
    return doc

#### A helper method that traverses through a county's website document and stores values for hourly wages

In [63]:
def getCounties(doc, state):
    #css paths to the website table
    county_css = '.counties div a'
    table_css = '.table-responsive'
    
    #for each county in the state
    for b in doc.cssselect(county_css):
        #populate lists for the dataframe
        state_names.append(state)
        county_name = b.text_content().strip()
        county_names.append(county_name)
        
        #reference to the website table    
        county_url = b.get('href')
        county_doc = scrapeSite(county_url)
        table = county_doc.cssselect(table_css)[0]
        col_heads = table.cssselect('thead tr')[0]
        
        #Loops through the rows for the 3-types of hourly wages
        wage_count = 0
        for row in table.cssselect('tbody tr'):
            col_head = col_heads.cssselect('th')[0].text_content().strip()
            row_head = row.cssselect('td')[1].text_content().strip()
            wage = row.cssselect('td')[1].text_content().strip()
            hourly_wages[wage_count].append( float(wage[1:]))
            wage_count += 1


#### Initialize multiple lists that will be used to create a Pandas Data Frame

In [64]:
#Prepping data for Pandas DataFrame
living_wages = []
poverty_wages = []
min_wages = []
hourly_wages = [living_wages, poverty_wages, min_wages]
county_names = []
state_names = []

### Part 2: Executing Web Scraper

In [71]:
state_css = ".states li a"
main_page = scrapeSite("http://livingwage.mit.edu")
state_names = []
for state in main_page.cssselect(state_css):
    
    #get the name of the state and start another scrape
    state_name = state.text_content()
    state_url = state.get('href')
    county_doc = scrapeSite(state_url)
    getCounties(county_doc, state_name)
    
    #show which state is finished being scraped
    print state_name


Alabama
Alaska
Arizona
Arkansas
California
Colorado
Connecticut
Delaware
District of Columbia
Florida
Georgia
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
Nevada
New Hampshire
New Jersey
New Mexico
New York
North Carolina
North Dakota
Ohio
Oklahoma
Oregon
Pennsylvania
Rhode Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Vermont
Virginia
Washington
West Virginia
Wisconsin
Wyoming


### Part 3: Saving Scraped Data

In [73]:
#construct a Pandas DataFrame using the lists created above
data = {'state': state_names,
        'county': county_names,
        'living': living_wages,
        'poverty': poverty_wages,
        'min': min_wages}

pandas_df = pandas.DataFrame(data) 
pandas_df.head(5)

,county,living,min,poverty,state
0,Autauga County,11.10,7.25,5.0,Alabama
1,Baldwin County,12.55,7.25,5.0,Alabama
2,Barbour County,10.54,7.25,5.0,Alabama
3,Bibb County,11.62,7.25,5.0,Alabama
4,Blount County,11.62,7.25,5.0,Alabama


In [74]:
#convert pandas dataframe to a csv file
pandas_df.to_csv('USA_Wages.csv')